In [ ]:
import collections

import networkx as nx
%matplotlib inline
import matplotlib.pyplot as plt

import parser
import imp; imp.reload(parser)

In [ ]:
# Unfortunately, we have to do this manually, since the condition is always logged as 'A' for some reason.

events1, level_sequence1 = parser.read_events("userlogs/p1")
events2, level_sequence2 = parser.read_events("userlogs/p2")
events3, level_sequence3 = parser.read_events("userlogs/p3")
events4, level_sequence4 = parser.read_events("userlogs/p4")
events5, level_sequence5 = parser.read_events("userlogs/p5")
events6, level_sequence6 = parser.read_events("userlogs/p6")
events7, level_sequence7 = parser.read_events("userlogs/p7")

blocks_only = level_sequence3 + level_sequence6
verbatim = level_sequence4 + level_sequence7
constructive = level_sequence2 + level_sequence5 + level_sequence1

### Vis.js graph visualization
Note: you *must* add vis.js to your static paths in your jupyter notebook config file. Please see [this tutorial](https://www.codementor.io/isaib.cicourel/visjs-visualization-in-jupyter-notebook-phgb3fjv0) for details.

*(Eventually we will wrap this module in a .js file and also add it to static paths, but for now the logic is not finished yet.)*

In [ ]:
# Convert some graph data into a JSON string
from networkx.readwrite import json_graph
import json
graph_to_json = lambda G: json.dumps(json_graph.node_link_data(G))
# test_data = graph_to_json(graphs[0])

merged = parser.mark_liveness(parser.get_complete_merged_graph(constructive, 25))
test_data = graph_to_json(merged)

# Pass graph data to JS backend
from IPython.display import Javascript
js = "window.graph_data={};".format(test_data)

# DO NOT ADD a ; to this line!! For some reason, it will not run it.
Javascript(js)

In [ ]:
js = ["window.conditions = {};"]
for condition, levels in [("blocks", blocks_only), ("verbatim", verbatim), ("constructive", constructive)]:
    js.append("window.conditions['{}'] = [];".format(condition))
    
    for level_idx in range(67):
        merged = parser.mark_liveness(parser.get_complete_merged_graph(levels, level_idx))
        merged_norm = parser.mark_liveness(parser.get_complete_merged_graph(levels, level_idx, do_normalize=True))
        js.append("window.conditions['{}'].push([{}, {}]);".format(
            condition,
            graph_to_json(merged),
            graph_to_json(merged_norm),
        ))
        
Javascript('\n'.join(js))

In [ ]:
js = ["window.temporal_conditions = {};"]
for condition, levels in [("blocks", blocks_only), ("verbatim", verbatim), ("constructive", constructive)]:
    js.append("window.temporal_conditions['{}'] = [];".format(condition))
    
    for level_idx in range(67):
        temporal = parser.mark_liveness(parser.temporal_graph(parser.only_complete_graphs(parser.get_complete_state_graphs(levels, level_idx))))
        temporal_norm = parser.mark_liveness(parser.temporal_graph(parser.only_complete_graphs(parser.get_complete_state_graphs(levels, level_idx, do_normalize=True))))
        js.append("window.temporal_conditions['{}'].push([{}, {}]);".format(
            condition,
            graph_to_json(temporal),
            graph_to_json(temporal_norm),
        ))
        
Javascript('\n'.join(js))

In [ ]:
%%javascript

/* Converts the networkx output (serialized through JSON) 
*  to vis.js format. */
function nx_to_vis(G) {
    
    // Convert nodes from nx format into vis.js format:
    var nodes = G["nodes"].map(function(n, idx) {
        var background = "#D2E5FF";
        if (n["terminal"] && n["live"]) {
            if (n["victory"])
                background = "#7fc97f";
            else
                background = "yellow";
        }
        else if (n["id"] == "prev" || n["id"] == "next") {
            background = "yellow";
        }
        else if (n["terminal"]) {
            background = "red";
        }
        else if (n["initial"]) {
            background = "#beaed4";
        }
        else if (!n["live"]) {
            background = "orange";
        }
        return { 
            id: idx, 
            label: n["id"], 
            color: {
                background: background,
            }
        };
    });
    
    // Convert edges ('links') to vis.js format:
    var edges = G["links"].map(function(e) {
        var r = { from: e["source"], to: e["target"] };
        if ("count" in e) r.label = "" + e.count;
        if ("weight" in e) {
            if (e.weight > 0.75) {
                r.width = 3;
            }
            else if (e.weight > 0.25) {
                r.width = 1;
            }
            else {
                r.width = 0.5;
            }
        }
        if ("live" in e) {
            if (!e.live) {
                r.color = "red";
            }
        }
        return r;
    });
    
    return {
        nodes: nodes,
        edges: edges
    };
}

var temporal_options = {
    width: '1000px',
    height: '800px',
    nodes: {
        shape: 'box',
        widthConstraint: 200,
        borderWidth: 0,
    },
    edges: {
        arrows: 'to',
        smooth: {
            type: 'cubicBezier',
            forceDirection: 'horizontal',
            roundness: 1,
        },
    },
    physics: false,
};

var default_options = {
    width: '800px',
    height: '800px',
    nodes: {
        shape: 'box'
    },
    edges: {
        arrows: 'to'
    }
};

function temporal_nx_to_vis(G) {
    
    // Convert nodes from nx format into vis.js format:
    var distance = 0;
    var cur_distance = 0;
    var cur_height = 0;
    var level = 0;
    
    var relative_height = false;
    
    var sort_on = function(key) {
        return function(a, b) {
            if (a[key] < b[key]) {
                return -1;
            }
            if (a[key] > b[key]) {
                return 1;
            }
            else {
                return 0;
            }
        };
    }
    
    var sorted_nodes = G["nodes"].map(function(node, idx) {
        node["idx"] = idx;
        return node;
    }).sort(function(a, b) {
        if (a["time"] < b["time"]) return -1;
        else if (a["time"] > b["time"]) return 1;
        return sort_on(relative_height ? "linear_height" : "height")(a, b);
    });
    
    var nodes = sorted_nodes.map(function(n) {
        var background = "#D2E5FF";
        if (n["terminal"] && n["live"]) {
            if (n["victory"])
                background = "#7fc97f";
            else
                background = "yellow";
        }
        else if (n["id"] == "prev" || n["id"] == "next") {
            background = "yellow";
        }
        else if (n["terminal"]) {
            background = "red";
        }
        else if (n["initial"]) {
            background = "#beaed4";
        }
        else if (!n["live"]) {
            background = "orange";
        }
        
        if (n["time"] != level) {
            level = n["time"];
            distance += cur_distance;
            cur_distance = 0;
            cur_height = 0;
        }
        
        cur_distance = Math.max(cur_distance, n["label"].length);
        cur_height += n["count"];
        return { 
            id: n["idx"], 
            label: n["label"], 
            color: {
                background: background,
                highlight: background,
            },
            x: n["time"] * 350,
            y: -(relative_height ? cur_height - 0.5*n["count"] : n["height"] * 1.25) * 20,
            heightConstraint: relative_height ? n["count"] * 15 : undefined,
        };
    });
    
    // Convert edges ('links') to vis.js format:
    var edges = G["links"].map(function(e) {
        var r = { from: e["source"], to: e["target"] };
        if ("count" in e) r.label = "" + e.count;
        if ("weight" in e) {
            if (e.weight > 0.75) {
                r.width = 3;
            }
            else if (e.weight > 0.25) {
                r.width = 1;
            }
            else {
                r.width = 0.5;
            }
        }
        if ("live" in e) {
            if (!e.live) {
                r.color = "orange";
            }
            if (e.reset) {
                r.color = "red";
            }
        }
        return r;
    });
    
    return {
        nodes: nodes,
        edges: edges
    };
}

require.config({
    paths: {
        vis: 'vis'
    }
});

require(['vis'], function(vis) {
    var refresh = function() {
        var use_temporal = temporal.prop('checked');
        var use_normalized = normalize.prop('checked');
        var graphs = use_temporal ? window.temporal_conditions : window.conditions;
        var graph_json = graphs[cond.val()][parseInt(level.val(), 10)][use_normalized ? 1 : 0];
        
        if ($("#display").length > 0) {
        // (a weird idempotency thing)
            $("#display").empty();
        }
        else {
        // Create an element to put the network into
            element.append("<div id='display' style='border:solid 1px'></div>");
        }

        // Decode it to vis.js format:
        var data = use_temporal ? temporal_nx_to_vis(graph_json) : nx_to_vis(graph_json);

        // Create the network
        var container = document.getElementById('display');
        var options = use_temporal ? temporal_options : default_options;
        var network = new vis.Network(container, data, options);
    };
    
    var cond = $("<select id='cond'></select>");
    var level = $("<select id='lvl'></select>");
    var temporal = $("<input type='checkbox'>");
    var normalize = $("<input type='checkbox'>");
    for (var condName in window.conditions) {
        cond.append($("<option>" + condName + "</option>"));
    }
    
    var changeCond = function() {
        var current = parseInt(level.val(), 10);
        level.empty();
        for (var i = 0; i < window.conditions[cond.val()].length; i++) {
            var levelopt = $("<option>" + i + "</option>");
            if (i == current) {
                levelopt.attr("selected", true);
            }
            level.append(levelopt);
        }
        refresh();
    };
    
    cond.on("change", changeCond);
    level.on("change", refresh);
    temporal.on("change", refresh);
    normalize.on("change", refresh);

    element.append(cond);
    element.append(level);
    element.append(temporal);
    element.append($("<span>Temporal graph</span>"));
    element.append(normalize);
    element.append($("<span>Normalized</span>"));
    
    changeCond();
});

In [ ]:
for condition, levels in [("blocks", blocks_only), ("verbatim", verbatim), ("constructive", constructive)]:
    for level_idx in range(67):
        merged = parser.mark_liveness(parser.get_complete_merged_graph(levels, level_idx))
        num_victory = len([None for node,data in merged.nodes(data=True) if data.get("victory")])
        if num_victory > 1:
            print(condition, level_idx)

In [ ]:
import imp; imp.reload(parser)
temporal = parser.mark_liveness(parser.temporal_graph(parser.only_complete_graphs(parser.get_complete_state_graphs(constructive, 41))))
js = "window.temporal={};".format(graph_to_json(temporal))

# DO NOT ADD a ; to this line!! For some reason, it will not run it.
Javascript(js)

In [ ]:
%%javascript

/* Converts the networkx output (serialized through JSON) 
*  to vis.js format. */
function nx_to_vis(G) {
    
    // Convert nodes from nx format into vis.js format:
    var distance = 0;
    var cur_distance = 0;
    var cur_height = 0;
    var level = 0;
    
    var relative_height = false;
    
    var sort_on = function(key) {
        return function(a, b) {
            if (a[key] < b[key]) {
                return -1;
            }
            if (a[key] > b[key]) {
                return 1;
            }
            else {
                return 0;
            }
        };
    }
    
    var sorted_nodes = G["nodes"].map(function(node, idx) {
        node["idx"] = idx;
        return node;
    }).sort(function(a, b) {
        if (a["time"] < b["time"]) return -1;
        else if (a["time"] > b["time"]) return 1;
        return sort_on(relative_height ? "linear_height" : "height")(a, b);
    });
    
    var nodes = sorted_nodes.map(function(n) {
        var background = "#D2E5FF";
        if (n["terminal"] && n["live"]) {
            if (n["victory"])
                background = "#7fc97f";
            else
                background = "yellow";
        }
        else if (n["id"] == "prev" || n["id"] == "next") {
            background = "yellow";
        }
        else if (n["terminal"]) {
            background = "red";
        }
        else if (n["initial"]) {
            background = "#beaed4";
        }
        else if (!n["live"]) {
            background = "orange";
        }
        
        if (n["time"] != level) {
            level = n["time"];
            distance += cur_distance;
            cur_distance = 0;
            cur_height = 0;
        }
        
        cur_distance = Math.max(cur_distance, n["label"].length);
        cur_height += n["count"];
        return { 
            id: n["idx"], 
            label: n["label"], 
            color: {
                background: background,
                highlight: background,
            },
            x: n["time"] * 350,
            y: -(relative_height ? cur_height - 0.5*n["count"] : n["height"] * 1.25) * 20,
            heightConstraint: relative_height ? n["count"] * 15 : undefined,
        };
    });
    
    // Convert edges ('links') to vis.js format:
    var edges = G["links"].map(function(e) {
        var r = { from: e["source"], to: e["target"] };
        if ("count" in e) r.label = "" + e.count;
        if ("weight" in e) {
            if (e.weight > 0.75) {
                r.width = 3;
            }
            else if (e.weight > 0.25) {
                r.width = 1;
            }
            else {
                r.width = 0.5;
            }
        }
        if ("live" in e) {
            if (!e.live) {
                r.color = "orange";
            }
            if (e.reset) {
                r.color = "red";
            }
        }
        return r;
    });
    
    return {
        nodes: nodes,
        edges: edges
    };
}

require.config({
    paths: {
        vis: 'vis'
    }
});

require(['vis'], function(vis) {
        var graph_json = window.temporal;
        
        if ($("#temporal-display").length > 0) {
        // (a weird idempotency thing)
            $("#temporal-display").empty();
        }
        else {
        // Create an element to put the network into
            element.append("<div id='temporal-display' style='border:solid 1px'></div>");
        }

        // Decode it to vis.js format:
        var data = nx_to_vis(graph_json);

        // Create the network
        var container = document.getElementById('temporal-display');
        var options = {
            width: '1000px',
            height: '800px',
            nodes: {
                shape: 'box',
                widthConstraint: 200,
                borderWidth: 0,
            },
            edges: {
                arrows: 'to',
                smooth: {
                    type: 'cubicBezier',
                    forceDirection: 'horizontal',
                    roundness: 1,
                },
            },
            physics: false,
        };
        var network = new vis.Network(container, data, options);

});

In [ ]:
%%javascript

/* Converts the networkx output (serialized through JSON) 
*  to vis.js format. */
function nx_to_vis(G) {
    
    // Convert nodes from nx format into vis.js format:
    var distance = 0;
    var cur_distance = 0;
    var level = 0;
    
    var sorted_nodes = G["nodes"].map(function(node, idx) {
        node["idx"] = idx;
        return node;
    }).sort(function(a, b) {
        if (a["time"] < b["time"]) return -1;
        else if (a["time"] > b["time"]) return 1;
        return 0;
    });
    
    var nodes = sorted_nodes.map(function(n) {
        var background = "#D2E5FF";
        if (n["terminal"] && n["live"]) {
            if (n["victory"])
                background = "#7fc97f";
            else
                background = "yellow";
        }
        else if (n["id"] == "prev" || n["id"] == "next") {
            background = "yellow";
        }
        else if (n["terminal"]) {
            background = "red";
        }
        else if (n["initial"]) {
            background = "#beaed4";
        }
        else if (!n["live"]) {
            background = "orange";
        }
        
        if (n["time"] != level) {
            level = n["time"];
            distance += cur_distance;
            cur_distance = 0;
        }
        
        cur_distance = Math.max(cur_distance, n["label"].length);
        return {
            level: n["time"],
            id: n["idx"], 
            label: n["label"], 
            color: {
                background: background,
            },
            borderWidth: 0,
        };
    });
    
    // Convert edges ('links') to vis.js format:
    var edges = G["links"].map(function(e) {
        var r = { from: e["source"], to: e["target"] };
        if ("count" in e) r.label = "" + e.count;
        if ("weight" in e) {
            if (e.weight > 0.75) {
                r.width = 3;
            }
            else if (e.weight > 0.25) {
                r.width = 1;
            }
            else {
                r.width = 0.5;
            }
        }
        if ("live" in e) {
            if (!e.live) {
                r.color = "red";
            }
        }
        return r;
    });
    
    return {
        nodes: nodes,
        edges: edges
    };
}

require.config({
    paths: {
        vis: 'vis'
    }
});

require(['vis'], function(vis) {
        var graph_json = window.temporal;
        var container = "temporal-display-2";
        
        if ($("#" + container).length > 0) {
        // (a weird idempotency thing)
            $("#" + container).empty();
        }
        else {
        // Create an element to put the network into
            element.append("<div id='" + container + "' style='border:solid 1px'></div>");
        }

        // Decode it to vis.js format:
        var data = nx_to_vis(graph_json);

        // Create the network
        var container = document.getElementById(container);
        var options = {
            width: '1000px',
            height: '800px',
            nodes: {
                shape: 'box'
            },
            edges: {
                arrows: 'to',
                smooth: {
                    type: 'cubicBezier',
                    forceDirection: 'horizontal',
                    roundness: 0.4,
                },
            },
            layout: {
                hierarchical: {
                    enabled: true,
                    direction: "LR",
                },
            },
            physics: false,
        };
        var network = new vis.Network(container, data, options);

});

In [ ]:
imp.reload(parser)

s = "[1, 2, true].map(x => (x + 1))"
a = esprima.parse(s, delegate=parser.NormalizeVisitor())
a

In [ ]:
parser.SerializeVisitor().visit(esprima.parse(s))

In [ ]:
parser.SerializeVisitor().visit(esprima.parse(s).body[0].expression.callee)

In [ ]:
esprima.parse(s).body[0].expression.callee.__class__

In [ ]:
parser.normalize(s)